## Quantium Internship Module 2 : Chips products data analysis

### Objective of the report

Our client is a company that owns several hypermarkets in the region that sells different types of products, in the previous notebook we received two datasets ***Transactions dataset** & **Customers dataset**  , we have conducted earlier an Exploratory data analysis on the two datasets and provided a conclusion of the business insights.

For this task Julia the Category Manager for Chips, has asked us to test the impact of the new trial layouts with a data driven recommendation to whether or not the trial layout should be rolled out to all their stores.

More specificaly she has asked us to evaluate the performance of a store trial which was performed in stores 77, 86 during the period of Feb2019 till April2019

In order to do so we have to define the **control stores** from our dataset Which are stores that are similar or close to the trial stores in the following metrics:

1. Total sales per month
2. Total customers per month

We will be examining the performance of control stores & compare it with the trial stores performance during the trial and provide a recommendation based on our insight.


Once we have selected the control stores, we will compare each trial and control pair during the trial period, we wil test if total sales & total number fo customers are significantly different in the trial period

In [287]:
# Import Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
import matplotlib.pyplot as plt
import calendar
from datetime import datetime
import scipy.stats as stats
%matplotlib inline
base_color=sns.color_palette()[9]

## Setting max displayed rows to 500, in order to display the full output of any command 
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)

# suppress warnings 
import warnings
warnings.simplefilter("ignore")

In [288]:
#importing data
df=pd.read_csv('QVI_data (1).csv')

In [289]:
#checking data first rows
df.head()

,LYLTY_CARD_NBR,DATE,STORE_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,PACK_SIZE,BRAND,LIFESTAGE,PREMIUM_CUSTOMER
0,1000,2018-10-17,1,1,5,Natural Chip Compny SeaSalt175g,2,6.0,175,NATURAL,YOUNG SINGLES/COUPLES,Premium
1,1002,2018-09-16,1,2,58,Red Rock Deli Chikn&Garlic Aioli 150g,1,2.7,150,RRD,YOUNG SINGLES/COUPLES,Mainstream
2,1003,2019-03-07,1,3,52,Grain Waves Sour Cream&Chives 210G,1,3.6,210,GRNWVES,YOUNG FAMILIES,Budget
3,1003,2019-03-08,1,4,106,Natural ChipCo Hony Soy Chckn175g,1,3.0,175,NATURAL,YOUNG FAMILIES,Budget
4,1004,2018-11-02,1,5,96,WW Original Stacked Chips 160g,1,1.9,160,WOOLWORTHS,OLDER SINGLES/COUPLES,Mainstream


We need to identify **control stores** as per the similarity between them and the trial stores (77,86) using the three metrics we decided to use:

1. Total sales per month
2. Total customers per month

**Features Engineering**

Extracting year and month columns from Date column

In [290]:
#Converting date to date type column
df['DATE']=pd.to_datetime(df['DATE'])

In [291]:
#Extracting year and month columns from Date column
df['Transaction_year']=df['DATE'].dt.year
df['Transaction_month']=df['DATE'].dt.month

Creating a year month column by concatenating year column with month column to make things easier in our analysis

In [292]:
#Creating a year month column
df['Yearmonth']=df['Transaction_year'].astype(str).str.zfill(2) + df['Transaction_month'].astype(str).str.zfill(3)

**Done now we will start summarizing data according to our metrics**


we will create a dataframe the has each store number along with the Total sales and Total customers per month

In [293]:
#creating an aggregated dataframe
Metricsdataframe=df[['STORE_NBR','Yearmonth','TOT_SALES','LYLTY_CARD_NBR']].groupby(['STORE_NBR','Yearmonth'],as_index=False).agg({'TOT_SALES':'sum','LYLTY_CARD_NBR':'count'}).rename(columns={'TOT_SALES':'Total Sales','LYLTY_CARD_NBR':'Number of customers per month'})
Metricsdataframe

,STORE_NBR,Yearmonth,Total Sales,Number of customers per month
0,1,2018007,206.9,52
1,1,2018008,176.1,43
2,1,2018009,278.8,62
3,1,2018010,188.1,45
4,1,2018011,192.6,47
...,...,...,...,...
3164,272,2019002,395.5,48
3165,272,2019003,442.3,53
3166,272,2019004,445.1,56
3167,272,2019005,314.6,40


Now let's consider which of the stores is more suitable to become a control store for 77,86, we will check the correlation between each store and the trial stores on the metrics that we created and the strongest correlation will be the best fit for a control store

In [294]:
# Create a function which calculates the correlation between trial store and other stores based on a single metric

def calculateCorrelation(inputTable, metric, trial_store):
    output = pd.DataFrame({'Store1': [], 'Store2': [], 'Correlation': []})
    a = inputTable.loc[inputTable['STORE_NBR'] == trial_store, metric]
    a.reset_index(drop = True, inplace = True)
    storeNumbers = inputTable['STORE_NBR'].unique()
    for i in storeNumbers:
        b = inputTable.loc[inputTable['STORE_NBR'] == i, metric]
        b.reset_index(drop = True, inplace = True)
        output = output.append({'Store1': trial_store, 'Store2': i, 'Correlation': b.corr(a)}, ignore_index = True)
    return output

**Calculating correlations between store 77 & other stores**

In [295]:
# Compute correlation with trial store 77
trial_store = 77
corr_nSales = calculateCorrelation(Metricsdataframe, 'Total Sales', trial_store)
corr_nCustomers = calculateCorrelation(Metricsdataframe, 'Number of customers per month', trial_store)

In [296]:
#checking total sales correlation
corr_nSales.rename(columns={'Correlation':'Total Sales correlation'},inplace=True)
corr_nSales.head()

,Store1,Store2,Total Sales correlation
0,77.0,1.0,0.064391
1,77.0,2.0,0.261602
2,77.0,3.0,0.162686
3,77.0,4.0,-0.294778
4,77.0,5.0,-0.284544


In [297]:
#checking total number of customers correlation
corr_nCustomers.rename(columns={'Correlation':'Number of customers correlation'},inplace=True)
corr_nCustomers.head()

,Store1,Store2,Number of customers correlation
0,77.0,1.0,0.331910
1,77.0,2.0,0.305525
2,77.0,3.0,0.405593
3,77.0,4.0,-0.221274
4,77.0,5.0,-0.322456


In [298]:
# Merging corr_nSales & corr_nCustomers into a finalscoredf77 dataset
finalscoredf77=corr_nSales.merge(corr_nCustomers,on=('Store1','Store2'),how='inner')
finalscoredf77.head()

,Store1,Store2,Total Sales correlation,Number of customers correlation
0,77.0,1.0,0.064391,0.331910
1,77.0,2.0,0.261602,0.305525
2,77.0,3.0,0.162686,0.405593
3,77.0,4.0,-0.294778,-0.221274
4,77.0,5.0,-0.284544,-0.322456


In [299]:
# Add a new column to 'finalscoredf77' which computes the average of 'Total Sales correlation' and 'Number of customers correlation'
finalscoredf77['finalScore'] = 0.5 * (finalscoredf77['Total Sales correlation'] + finalscoredf77['Number of customers correlation'])
finalscoredf77.sort_values(by='finalScore',ascending=False).head()

,Store1,Store2,Total Sales correlation,Number of customers correlation,finalScore
76,77.0,77.0,1.000000,1.000000,1.000000
34,77.0,35.0,0.699708,0.811399,0.755553
40,77.0,41.0,0.762292,0.619170,0.690731
166,77.0,167.0,0.696075,0.683172,0.689624
233,77.0,234.0,0.632204,0.741529,0.686866


Now we have concluded that the best final score is store 64 which makes it the control store for trial store 77

**Assessing the difference in Sales & Total customers between two stores during the period of the trial for 77 and 64 stores**

In [301]:
df77=df[df['STORE_NBR']==77]
df77=df77[['STORE_NBR','Yearmonth','TOT_SALES','LYLTY_CARD_NBR']].groupby(['STORE_NBR','Yearmonth'],as_index=False).agg({'TOT_SALES':'sum','LYLTY_CARD_NBR':'count'})
df77.rename(columns={'TOT_SALES':'Store 77 Total Sales','STORE_NBR':'Store 77','LYLTY_CARD_NBR':'store 77 total customers'},inplace=True)
df35=df[df['STORE_NBR']==35]
df35=df35[['STORE_NBR','Yearmonth','TOT_SALES','LYLTY_CARD_NBR']].groupby(['STORE_NBR','Yearmonth'],as_index=False).agg({'TOT_SALES':'sum','LYLTY_CARD_NBR':'count'})
df35.rename(columns={'TOT_SALES':'Store 35 Total Sales','STORE_NBR':'Store 35','LYLTY_CARD_NBR':'store 35 total customers'},inplace=True)

Joinning the two datasets together to compare difference is sales during trial period and calculating a percentage change the sales to report accurate figures

In [262]:
#joinning dataframes
salescomparison_df=df77.merge(df35,on=('Yearmonth'),how='inner')

# creating a percentage change column for sales and total customers columns
salescomparison_df['sales percentageDiff']=(salescomparison_df['Store 77 Total Sales']-salescomparison_df['Store 35 Total Sales'])/salescomparison_df['Store 77 Total Sales']
salescomparison_df['customers percentageDiff']=(salescomparison_df['store 77 total customers']-salescomparison_df['store 35 total customers'])/salescomparison_df['store 77 total customers']

#choosing specific features to show 
salescomparison_df=salescomparison_df[['Yearmonth','Store 77 Total Sales','Store 35 Total Sales','sales percentageDiff','store 77 total customers','store 35 total customers','customers percentageDiff']]
salescomparison_df

,Yearmonth,Store 77 Total Sales,Store 35 Total Sales,sales percentageDiff,store 77 total customers,store 35 total customers,customers percentageDiff
0,2018007,296.8,124.2,0.581536,55,37,0.327273
1,2018008,255.5,147.6,0.422309,48,40,0.166667
2,2018009,225.2,121.3,0.461368,44,32,0.272727
3,2018010,204.5,128.1,0.373594,38,33,0.131579
4,2018011,245.3,121.2,0.505911,44,34,0.227273
5,2018012,267.3,147.1,0.449682,49,37,0.244898
6,2019001,204.4,99.5,0.513209,39,27,0.307692
7,2019002,235.0,117.2,0.501277,45,32,0.288889
8,2019003,278.5,156.4,0.438420,55,39,0.290909
9,2019004,263.5,143.6,0.455028,48,40,0.166667


**Observations:**There is an obvious increase in sales & number of customers during the trial period for store 77 in February,March,April 2019 compared to control stores in the same period

**Calculating correlations between store 86 & other stores**

In [263]:
# Compute correlation with trial store 86
trial_store = 86
corr_nSales = calculateCorrelation(Metricsdataframe, 'Total Sales', trial_store)
corr_nCustomers = calculateCorrelation(Metricsdataframe, 'Number of customers per month', trial_store)

In [264]:
#checking total sales correlation
corr_nSales.rename(columns={'Correlation':'Total Sales correlation'},inplace=True)
corr_nSales.head()

,Store1,Store2,Total Sales correlation
0,86.0,1.0,0.343541
1,86.0,2.0,0.001103
2,86.0,3.0,0.178180
3,86.0,4.0,-0.179046
4,86.0,5.0,-0.321841


In [265]:
#checking total number of customers correlation
corr_nCustomers.rename(columns={'Correlation':'Number of customers correlation'},inplace=True)
corr_nCustomers.head()

,Store1,Store2,Number of customers correlation
0,86.0,1.0,0.287938
1,86.0,2.0,-0.045875
2,86.0,3.0,-0.006308
3,86.0,4.0,-0.241772
4,86.0,5.0,-0.618249


In [266]:
# Merging corr_nSales & corr_nCustomers into a finalscore dataset
finalscoredf86=corr_nSales.merge(corr_nCustomers,on=('Store1','Store2'),how='inner')
finalscoredf86.head()

,Store1,Store2,Total Sales correlation,Number of customers correlation
0,86.0,1.0,0.343541,0.287938
1,86.0,2.0,0.001103,-0.045875
2,86.0,3.0,0.178180,-0.006308
3,86.0,4.0,-0.179046,-0.241772
4,86.0,5.0,-0.321841,-0.618249


In [267]:
# Add a new column to 'finalscoredf86' which computes the average of 'Total Sales correlation' and 'Number of customers correlation'
finalscoredf86['finalScore'] = 0.5 * (finalscoredf86['Total Sales correlation'] + finalscoredf86['Number of customers correlation'])
finalscoredf86.sort_values(by='finalScore',ascending=False).head()

,Store1,Store2,Total Sales correlation,Number of customers correlation,finalScore
85,86.0,86.0,1.000000,1.000000,1.000000
146,86.0,147.0,0.544236,0.769144,0.656690
230,86.0,231.0,0.674071,0.601186,0.637629
60,86.0,61.0,0.617243,0.600191,0.608717
98,86.0,99.0,0.576043,0.640161,0.608102


Now we have concluded that the best final score is store 147 which makes it the control store for trial store 86

**Assessing the difference in Sales & Total customers between two stores during the period of the trial for 86 and 147 stores**

In [268]:
df86=df[df['STORE_NBR']==86]
df86=df86[['STORE_NBR','Yearmonth','TOT_SALES','LYLTY_CARD_NBR']].groupby(['STORE_NBR','Yearmonth'],as_index=False).agg({'TOT_SALES':'sum','LYLTY_CARD_NBR':'count'})
df86.rename(columns={'TOT_SALES':'Store 86 Total Sales','STORE_NBR':'Store 86','LYLTY_CARD_NBR':'store 86 total customers'},inplace=True)
df147=df[df['STORE_NBR']==147]
df147=df147[['STORE_NBR','Yearmonth','TOT_SALES','LYLTY_CARD_NBR']].groupby(['STORE_NBR','Yearmonth'],as_index=False).agg({'TOT_SALES':'sum','LYLTY_CARD_NBR':'count'})
df147.rename(columns={'TOT_SALES':'Store 147 Total Sales','STORE_NBR':'Store 147','LYLTY_CARD_NBR':'store 147 total customers'},inplace=True)

Joinning the two datasets together to compare difference is sales during trial period and calculating a percentage change the sales to report accurate figures

In [270]:
#joinning dataframes
salescomparison_df=df86.merge(df147,on=('Yearmonth'),how='inner')

# creating a percentage change column for sales and total customers columns
salescomparison_df['sales percentageDiff']=(salescomparison_df['Store 86 Total Sales']-salescomparison_df['Store 147 Total Sales'])/salescomparison_df['Store 86 Total Sales']
salescomparison_df['customers percentageDiff']=(salescomparison_df['store 86 total customers']-salescomparison_df['store 147 total customers'])/salescomparison_df['store 86 total customers']

#choosing specific features to show 
salescomparison_df=salescomparison_df[['Yearmonth','Store 86 Total Sales','Store 147 Total Sales','sales percentageDiff','store 86 total customers','store 147 total customers','customers percentageDiff']]
salescomparison_df

,Yearmonth,Store 86 Total Sales,Store 147 Total Sales,sales percentageDiff,store 86 total customers,store 147 total customers,customers percentageDiff
0,2018007,892.20,755.60,0.153105,126,109,0.134921
1,2018008,764.05,674.45,0.117270,112,106,0.053571
2,2018009,914.60,785.80,0.140827,129,116,0.100775
3,2018010,948.40,952.60,-0.004429,138,139,-0.007246
4,2018011,918.00,788.60,0.140959,127,118,0.070866
5,2018012,841.20,878.20,-0.043985,120,120,0.000000
6,2019001,841.40,860.00,-0.022106,130,120,0.076923
7,2019002,913.20,881.80,0.034385,139,127,0.086331
8,2019003,1026.80,881.80,0.141215,142,124,0.126761
9,2019004,848.20,866.00,-0.020986,127,124,0.023622


**observations:** There was'nt a significant change in total sales per month in February and march and there was a decrease in sales in April for store 86 while there was a slight increase in the Total customers per month for the whole trial period

**Conclusion:** We had an increase in sales & total customers for store 77 during the trial period but for store 86 There was'nt a significant change in total sales per month in February and march and there was a decrease in sales in April while there was a slight increase in the Total customers per month for the whole trial period.